План проекта
1. Отобрать 9 популяций:
- Africa (393)
    - yoruba
    - bantu
    - egyptans
- Asia (502)
    - indians
    - japanese
    - chinese
- Europe (390)
    - finns
    - english
    - italians

2. Выбрать базы (Поскольку образцы повторяются от базы к базе) содержащую макс кол-во образцов - потом максимум от того что осталось итд.
3. Перевести в бинарник, если необходимо.
4. Выровнять на последнюю версию генома.
5. Слить в один бинарный файл. 
6. Убрать лишние SNP.
7. Проверить пересечение SNP, отбирать там где больше 300 000. 
8. Установить семью как популяцию.
9. Сделать PCA на 10 первых компонент. Картинка с цветами и семьями.
10. Прогнать admixture. Сделать картинку на популяции в admixture

In [ ]:
!wget http://s3.amazonaws.com/plink1-assets/plink_linux_x86_64_20200219.zip
!unzip plink_linux_x86_64_20200219.zip -d plink/

In [ ]:
#!pip install pandas

In [1]:
import pandas as pd
#import sys
#from os import listdir
#from os.path import isfile, join
#import codecs
import numpy as np

In [2]:
df = pd.read_csv('Databases - individuals.csv')

In [3]:
df

,ID,population,Subpopulation,Region Macro,Region,Accuracy of pop,Project,Type of sequencing,directory,downloaded,Unnamed: 10,Unnamed: 11
0,Hazara6_27Af,hazaras,NaN,Asia,Asia Central,NaN,Estonian Biocenter,GENOTYPE,Estonian_Biocenter/afgan/,yes,NaN,NaN
1,Hazara6_2Af,hazaras,NaN,Asia,Asia Central,NaN,Estonian Biocenter,GENOTYPE,Estonian_Biocenter/afgan/,yes,NaN,NaN
2,Hazara6_66Af,hazaras,NaN,Asia,Asia Central,NaN,Estonian Biocenter,GENOTYPE,Estonian_Biocenter/afgan/,yes,NaN,NaN
3,Hazara6_68Af,hazaras,NaN,Asia,Asia Central,NaN,Estonian Biocenter,GENOTYPE,Estonian_Biocenter/afgan/,yes,NaN,NaN
4,Hazara6_6Af,hazaras,NaN,Asia,Asia Central,NaN,Estonian Biocenter,GENOTYPE,Estonian_Biocenter/afgan/,yes,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
12223,SGVP00598,indians,NaN,Asia,Asia Southeast,NaN,Singapore Genome Variation Project,NaN,SingaporeGVP/2009-0-30/,yes,NaN,NaN
12224,SGVP00015,chinese,NaN,Asia,Asia Southeast,NaN,Singapore Genome Variation Project,NaN,SingaporeGVP/2009-0-30/,yes,NaN,NaN
12225,SGVP00016,chinese,NaN,Asia,Asia Southeast,NaN,Singapore Genome Variation Project,NaN,SingaporeGVP/2009-0-30/,yes,NaN,NaN
12226,SGVP00178,chinese,NaN,Asia,Asia Southeast,NaN,Singapore Genome Variation Project,NaN,SingaporeGVP/2009-0-30/,yes,NaN,NaN


In [4]:
listofpopulations = ['bantu', 'yoruba', 'egyptans', 'Egyptian', 'Indian', 'indians', 'chinese', 'japanese', 'finns', 'english', 'italians']

In [5]:
id = [y for x in listofpopulations for y in df.ID[df.population == x]]
dr = [y for x in listofpopulations for y in df.directory[df.population == x]]
print (id)
print(dr)

['HGDP01405', 'HGDP01408', 'HGDP01413', 'HGDP01416', 'HGDP01419', 'HGDP00993', 'HGDP00994', 'HGDP01028', 'HGDP01030', 'HGDP01031', 'HGDP01033', 'HGDP01034', 'HGDP01035', 'HGDP01405', 'HGDP01406', 'HGDP01408', 'HGDP01411', 'HGDP01412', 'HGDP01413', 'HGDP01414', 'HGDP01415', 'HGDP01416', 'HGDP01417', 'HGDP01418', 'HGDP01419', 'HGDP00993', 'HGDP00994', 'HGDP01028', 'HGDP01030', 'HGDP01031', 'HGDP01033', 'HGDP01034', 'HGDP01035', 'HGDP01405', 'HGDP01406', 'HGDP01408', 'HGDP01411', 'HGDP01412', 'HGDP01413', 'HGDP01414', 'HGDP01415', 'HGDP01416', 'HGDP01417', 'HGDP01418', 'HGDP01419', 'HGDP01028', 'HGDP01030', 'HGDP01034', 'HGDP01035', 'HGDP01414', 'HGDP01417', 'HGDP00993', 'HGDP00994', 'HGDP01028', 'HGDP01030', 'HGDP01031', 'HGDP01033', 'HGDP01034', 'HGDP01035', 'HGDP01405', 'HGDP01406', 'HGDP01408', 'HGDP01411', 'HGDP01412', 'HGDP01414', 'HGDP01415', 'HGDP01416', 'HGDP01417', 'HGDP01418', 'HGDP01419', 'HGDP00927', 'SS6004475', 'HGDP00921', 'HGDP00928', 'HGDP00932', 'HGDP00938', 'HGDP00942'

In [6]:
len(id)

1282

**Распределение популяций по папкам**

In [7]:
drct = df[df.population.isin(listofpopulations)].groupby(by = 'directory').count().idxmax().ID
unique_dict = {}
best_count = 1
unique_dict = {x:y for x, y in zip(id, dr) if y == drct}
samples_in_directories =[[drct, len(unique_dict)]]
#unique_list(collect_sample)
cnt = 0
while  best_count > 0:
    dir_list = []
    for drct in df.directory[df.population.isin(listofpopulations)].unique():

        dir_list.append([drct, len([sample for sample in df.ID[df.population.isin(listofpopulations) & (df.directory == drct)] if sample not in unique_dict.keys()])])
        #print ('1 - ', drct, ' = ', len([sample for sample in df.ID[df.population.isin(listofpopulations) & (df.directory == drct)] if sample not in unique_dict.keys()]))
    best_count = max([int(x[1]) for x in dir_list])
    best_dir = [y[0] for y in dir_list if y[1] == best_count][0]
    if best_count ==0:
        break
    unique_dict.update({x:y for x, y in zip(id, dr) if (y == best_dir) & (x not in unique_dict)})
    samples_in_directories.append([best_dir, best_count])
    #print('6 - next step', best_dir, best_count)
    cnt +=1
    if cnt > 50:
        break

for record in samples_in_directories:
    print(record)

['ReichLab/AncietPresent2019/long', 558]
['SingaporeGVP/2009-0-30/', 179]
['ReichLab/AncietPresent2019/short', 152]
['SIBERIA/', 46]
['Estonian_Biocenter/Tamm_2019', 34]
['Estonian_Biocenter/khazar', 24]
['Jakobsson2008/', 20]
['Estonian_Biocenter/Tambets2018', 19]
['Estonian_Biocenter/jew/', 4]
['Estonian_Biocenter/Eichstaedt_2017', 3]
['Li2008/', 2]
['Estonian_Biocenter/america/', 1]


1. Готовим данные из папки Recihlab. Появилась новая 42 версия датасета (у нас 37). Обе версии выравненны на hg19 или 37 версию генома. Конвертация из формата проекта в бинарник происходит софтом eigensoft. 

Устанавливаем eigensof

In [ ]:
!sudo apt-get install eigensoft

Закачиваем новую версию датасета

In [ ]:
!wget https://reichdata.hms.harvard.edu/pub/datasets/amh_repo/curated_releases/V42/V42.4/SHARE/public.dir/v42.4.1240K.tar

In [ ]:
!mkdir -p /home/cappelchi/ReichLab/V42/long/
!tar -xf v42.4.1240K.tar -C /home/cappelchi/ReichLab/V42/long

Готовим файл для конвертации

In [ ]:
!echo "genotypename:    /home/cappelchi/ReichLab/V42/long/v42.4.1240K.geno" > /home/cappelchi/ReichLab/V42/long/par.EIGENSTRAT.PEDPACKED
!echo "snpname:        /home/cappelchi/ReichLab/V42/long/v42.4.1240K.snp" >> /home/cappelchi/ReichLab/V42/long/par.EIGENSTRAT.PEDPACKED
!echo "indivname:       /home/cappelchi/ReichLab/V42/long/v42.4.1240K.ind" >> /home/cappelchi/ReichLab/V42/long/par.EIGENSTRAT.PEDPACKED
!echo "outputformat:    PACKEDPED" >> /home/cappelchi/ReichLab/V42/long/par.EIGENSTRAT.PEDPACKED
!echo "genotypeoutname: /home/cappelchi/ReichLab/V42/long/v42.4.1240K.bed" >> /home/cappelchi/ReichLab/V42/long/par.EIGENSTRAT.PEDPACKED
!echo "snpoutname:      /home/cappelchi/ReichLab/V42/long/v42.4.1240K.bim" >> /home/cappelchi/ReichLab/V42/long/par.EIGENSTRAT.PEDPACKED
!echo "indivoutname:    /home/cappelchi/ReichLab/V42/long/v42.4.1240K.fam" >> /home/cappelchi/ReichLab/V42/long/par.EIGENSTRAT.PEDPACKED
!echo "familynames:     NO" >> /home/cappelchi/ReichLab/V42/long/par.EIGENSTRAT.PEDPACKED
!cat /home/cappelchi/ReichLab/V42/long/par.EIGENSTRAT.PEDPACKED

In [ ]:
!convertf -p /home/cappelchi/ReichLab/V42/long/par.EIGENSTRAT.PEDPACKED

.fam files specification:
Family ID ('FID')
Within-family ID ('IID'; cannot be '0')
Within-family ID of father ('0' if father isn't in dataset)
Within-family ID of mother ('0' if mother isn't in dataset)
Sex code ('1' = male, '2' = female, '0' = unknown)
Phenotype value ('1' = control, '2' = case, '-9'/'0'/non-numeric = missing data if case/control)

In [8]:
df_fam = pd.read_csv(
    '/home/cappelchi/ReichLab/V42/long/v42.4.1240K.fam', 
    sep = '\s+', 
    names = ['ID1', 'ID2', 'IDF', 'IDM', 'sex', 'pop'], 
    header = None, 
    index_col = False
    )

Собираем названия образцов для выборки

In [9]:
Reich_long = [key for key in unique_dict if unique_dict[key] == 'ReichLab/AncietPresent2019/long']

In [10]:
Reich_long

['HGDP01028',
 'HGDP01030',
 'HGDP01034',
 'HGDP01035',
 'HGDP01414',
 'HGDP01417',
 'A_Yoruba-4',
 'HGDP00928',
 'HGDP00932',
 'HGDP00936',
 'NA18486',
 'NA18488',
 'NA18489',
 'NA18498',
 'NA18499',
 'NA18501',
 'NA18502',
 'NA18504',
 'NA18505',
 'NA18507',
 'NA18508',
 'NA18510',
 'NA18511',
 'NA18516',
 'NA18517',
 'NA18519',
 'NA18520',
 'NA18522',
 'NA18523',
 'NA18853',
 'NA18856',
 'NA18858',
 'NA18861',
 'NA18864',
 'NA18865',
 'NA18867',
 'NA18868',
 'NA18870',
 'NA18871',
 'NA18873',
 'NA18874',
 'NA18876',
 'NA18877',
 'NA18878',
 'NA18879',
 'NA18881',
 'NA18907',
 'NA18908',
 'NA18909',
 'NA18910',
 'NA18912',
 'NA18915',
 'NA18916',
 'NA18917',
 'NA18923',
 'NA18924',
 'NA18933',
 'NA18934',
 'NA19092',
 'NA19093',
 'NA19095',
 'NA19096',
 'NA19098',
 'NA19099',
 'NA19102',
 'NA19107',
 'NA19108',
 'NA19113',
 'NA19114',
 'NA19116',
 'NA19117',
 'NA19118',
 'NA19119',
 'NA19121',
 'NA19129',
 'NA19130',
 'NA19131',
 'NA19137',
 'NA19138',
 'NA19141',
 'NA19143',
 'NA191

Оказалось, что в новой базе названия образцов немного изменились. Корректируем названия для вырезания из датасета.

In [11]:
corr_names = [name for name in df_fam.ID2 if (name in Reich_long) or (name.split('.')[0] in Reich_long)]
print(len(corr_names))

531


Количество образцов немного уменьшилось. Смотрим какие образцы не вошли.

In [12]:
split_point = [name.split('.')[0] for name in df_fam.ID2 if (name in Reich_long) or (name.split('.')[0] in Reich_long)]
print(len(split_point))
rest = [name for name in Reich_long if name not in split_point]
print (rest)

531
['HGDP01028', 'HGDP01030', 'HGDP01034', 'HGDP01035', 'HGDP01414', 'HGDP01417', 'HGDP00928', 'HGDP00932', 'HGDP00936', 'IL-01', 'IL-04', 'IL-07', 'IL-08', 'IL-09', 'IL-10', 'IL-11', 'IL-12', 'IL-15', 'IL-16', 'HGDP00749', 'HGDP00773', 'HGDP01076', 'HGDP01078', 'HGDP01079', 'HGDP01163', 'HGDP01168', 'HGDP01172']


Апдейтим датасет под нашу выборку

In [ ]:
df_fam_selected = df_fam[df_fam.ID2.isin(corr_names)]

In [ ]:
df_fam_selected.to_csv('/home/cappelchi/ReichLab/V42/long/keep.fam',
                      sep = ' ',
                       header = False,
                       index = False                       
                      )

In [ ]:
!plink/plink --bfile /home/cappelchi/ReichLab/V42/long/v42.4.1240K --keep-fam /home/cappelchi/ReichLab/V42/long/keep.fam --make-bed --out /home/cappelchi/ReichLab/V42/long/v42.4.1240K.selected

In [ ]:
!plink/plink --bfile /home/cappelchi/ReichLab/V42/long/v42.4.1240K.selected  --keep-allele-order --recode vcf --out /home/cappelchi/ReichLab/V42/long/v42.4.1240K.selected

In [22]:
df_vcf = pd.read_csv('/home/cappelchi/ReichLab/V42/long/v42.4.1240K.selected.vcf', 
                     #names = ['CHR', 'SNP', ' gen_dist', ' pos', 'ref', 'ALT'], 
                     sep = '\t', 
                     header = 29, 
                     index_col = False,
                     nrows = 1000
                    )
df_vcf = df_vcf.set_index(df_vcf.ID)

In [ ]:
!pip install Biopython

In [16]:
from Bio import Entrez

In [13]:
def get_words(words):
    collect = {}
    for word in words.split('><'):
        if 'DocumentSummary uid=' in word:
            collect.update({'uid':word.split('"')[1]})
        elif ('>' in word) & ('DocumentSummary' not in word):
            collect.update({word.split('>')[0]:word.split('>')[1].split('<')[0]})
    return collect

In [14]:
def get_response(response):
#'ACC': 'NC_000022.11',
# 'ALLELE': 'K',
# 'CHR': '22',
# 'CHRPOS': '22:16586614',
# 'CHRPOS_PREV_ASSM': '22:17067504',
# 'CHRPOS_SORT': '0016586614',
# 'CLINICAL_SORT': '0',
# 'CREATEDATE': '2003/04/21 10:30',
# 'DOCSUM': 'HGVS=NC_000022.11:g.16586614G&gt;T,NC_000022.10:g.17067504G&gt;T|SEQ=[G/T]',
# 'FREQ': 'G=0.380952/80',
# 'GLOBAL_SAMPLESIZE': '0',
# 'HANDLE': 'BGI,CSHL,PJP,PERLEGEN,HUMANGENOME_JCVI,GRF,EVA_SVP,COMPLETE_GENOMICS,BIOINF_KMB_FNS_UNIBA,SWEGEN,CSHL-HAPMAP,WI_SSAHASNP,WEILL_CORNELL_DGM,SC_SNP,1000GENOMES,ILLUMINA,SSMP,EVA-GONL,ACPOP,EVA_UK10K_ALSPAC,EVA,BL,BUSHMAN,HAMMER_LAB,TOPMED,KHV_HUMAN_GENOMES,ABI,KRIBB_YJKIM,GMI,ENSEMBL,EVA_UK10K_TWINSUK,URBANLAB,BCM_SSAHASNP,GNOMAD,SYSTEMSBIOZJU,SSAHASNP,HGSV,DDI,JJLAB',
# 'MERGED_SORT': '0',
# 'ORIG_BUILD': '114',
# 'SNP_CLASS': 'snv',
# 'SNP_ID': '5747968',
# 'SNP_ID_SORT': '0005747968',
# 'SS': '7993836,10994773,12519402,20142689,21839947,24541363,41531468,67419767,67781042,68244159,70846723,71431451,75581182,77796551,80635583,84422108,96112946,106217979,139910456,154340557,159517140,167641816,173926858,204043240,255828096,283575927,287545757,292731800,341071521,537313750,662465704,833028430,995199824,1366598733,1429211946,1639715074,1682709107,1713729499,1809723698,1938758222,2030152705,2413108330,2629574271,2635110549,2704498642,2972723071,3019040464,3028914573,3352763058,3373459792,3628500813,3638372909,3639190399,3639897420,3643333452,3643944839,3651146196,3743802868,3759204350,3822370344',
# 'STUDY': 'Vietnamese',
# 'TAX_ID': '9606',
# 'UPDATEDATE': '2019/07/14 03:14',
# 'UPD_BUILD': '153',
# 'VALIDATED': 'by-frequency,by-cluster',
# 'uid': '5747968'
    dict_rsps ={}
    lines = [line for line in response.split('\n')]
    for words in lines:
        if words != '':
            rsp = get_words(words)
            dict_rsps.update({'rs' + str(rsp['uid']):rsp})
    return dict_rsps

In [17]:
Entrez.email = 'cappelchi@gmail.com'

In [24]:
snp_list = [snp for snp in df_vcf.ID[0:7]]
response = Entrez.efetch(db='SNP', id=','.join(snp_list), rettype='flt', retmode='flt').read()
dict_response = get_response(response)
for rs in snp_list:
    print(rs, ' - ', dict_response[rs]['DOCSUM'].split(dict_response[rs]['CHRPOS'].split(':')[1])[1].split('&gt;')[0],
        ' - ', dict_response[rs]['DOCSUM'].split(dict_response[rs]['CHRPOS'].split(':')[1])[1].split('&gt;')[1].split(',')[0],
        ' - newPOS: ', dict_response[rs]['CHRPOS'].split(':')[1], ' - oldPOS: ', df_vcf.POS[rs]
        )

rs3094315  -  G  -  A  - newPOS:  817186  - oldPOS:  752566
rs12124819  -  A  -  G  - newPOS:  841166  - oldPOS:  776546
rs28765502  -  T  -  C  - newPOS:  897538  - oldPOS:  832918
rs7419119  -  T  -  G  - newPOS:  906633  - oldPOS:  842013
rs950122  -  G  -  C  - newPOS:  911484  - oldPOS:  846864
rs113171913  -  C  -  T  - newPOS:  933923  - oldPOS:  869303
rs13302957  -  G  -  A  - newPOS:  955641  - oldPOS:  891021


In [51]:
!pip install tqdm

    100% |████████████████████████████████| 61kB 1.5MB/s ta 0:00:011


In [20]:
!pip install pyliftover

  Stored in directory: /home/cappelchi/.cache/pip/wheels/da/43/4f/f7393d2ee376f5ef5c019ff32c145b1c2f26b52bd661addc58
Successfully built pyliftover


In [21]:
import os
import csv
from tqdm import tqdm
from pyliftover import LiftOver

In [23]:
lo = LiftOver('hg19ToHg38.over.chain.gz')

In [101]:
with open('/home/cappelchi/ReichLab/V42/long/v42.4.1240K.selected.vcf', 'r') as input_file:
    reader = csv.reader(input_file, delimiter = '\t')
    trigger = False
    cnt = 0
    cnt_crd = 0
    cnt_mis = 0
    with open('/home/cappelchi/ReichLab/V42/long/v42.4.1240K.selected_v38.vcf', 'a') as output_file:
        for row in tqdm(reader):
            
            #if cnt > 3:
            #    break
            writer = csv.writer(output_file, delimiter = '\t')
            if trigger:
                cnt += 1
                #print('chr' + str(row[0]), row[1])
                rw = lo.convert_coordinate('chr' + str(row[0]), int(row[1]))
                if rw:                   
                    row[1] = rw[0][1]
                    #print(lo.convert_coordinate('chr' + str(row[0]), row[1])[0][1])
                    writer.writerow(row)
                    cnt_crd += 1
                else:
                    cnt_mis +=1
            #else:
            #    writer.writerow(row)
            if row[0] == '#CHROM':
                writer.writerow(row)
                trigger = True

print ('Cнипов перемещено = ', cnt_crd)
print ('Снипов не найдено = ', cnt_mis)

1233043it [02:39, 7710.38it/s] 

Cнипов перемещено =  1150565
Снипов не найдено =  82448


In [110]:
!plink/plink --vcf /home/cappelchi/ReichLab/V42/long/v42.4.1240K.selected_v38.vcf --double-id --make-bed --out /home/cappelchi/ReichLab/V42/long/v42.4.1240K.selected_v38

PLINK v1.90b6.16 64-bit (19 Feb 2020)          www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /home/cappelchi/ReichLab/V42/long/v42.4.1240K.selected_v38.log.
Options in effect:
  --double-id
  --make-bed
  --out /home/cappelchi/ReichLab/V42/long/v42.4.1240K.selected_v38
  --vcf /home/cappelchi/ReichLab/V42/long/v42.4.1240K.selected_v38.vcf

3944 MB RAM detected; reserving 1972 MB for main workspace.
--vcf: /home/cappelchi/ReichLab/V42/long/v42.4.1240K.selected_v38-temporary.bed
+ /home/cappelchi/ReichLab/V42/long/v42.4.1240K.selected_v38-temporary.bim +
/home/cappelchi/ReichLab/V42/long/v42.4.1240K.selected_v38-temporary.fam
written.
1150565 variants loaded from .bim file.
531 people (0 males, 0 females, 531 ambiguous) loaded from .fam.
Ambiguous sex IDs written to
/home/cappelchi/ReichLab/V42/long/v42.4.1240K.selected_v38.nosex .
Using 1 thread (no multithreaded calculations invoked).
Before main variant filter

In [108]:
!head -35 /home/cappelchi/ReichLab/V42/long/v42.4.1240K.selected_v38.vcf

In [99]:
!head -35 /home/cappelchi/ReichLab/V42/long/v42.4.1240K.selected_v38.vcf

PLINK v1.90b6.16 64-bit (19 Feb 2020)          www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /home/cappelchi/ReichLab/V42/long/v42.4.1240K.selected_v38.log.
Options in effect:
  --id-delim
  --make-bed
  --out /home/cappelchi/ReichLab/V42/long/v42.4.1240K.selected_v38
  --vcf



In [68]:
!plink/plink --vcf /home/cappelchi/ReichLab/V42/long/v42.4.1240K.selected_v38.vcf --make-bed --out /home/cappelchi/ReichLab/V42/long/v42.4.1240K.selected_v38

PLINK v1.90b6.16 64-bit (19 Feb 2020)          www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /home/cappelchi/ReichLab/V42/long/v42.4.1240K.selected_v38.log.
Options in effect:
  --make-bed
  --out /home/cappelchi/ReichLab/V42/long/v42.4.1240K.selected_v38
  --vcf /home/cappelchi/ReichLab/V42/long/v42.4.1240K.selected_v38.vcf

3944 MB RAM detected; reserving 1972 MB for main workspace.
Error: Missing header line in .vcf file.


In [ ]:
chunk = 1000
with open('/home/cappelchi/ReichLab/V42/long/v42.4.1240K.selected.vcf', 'r') as input_file:
    cnt = 0
    #for line in csvfile:
    #    print(line)
        
        
    reader = csv.reader(input_file, delimiter = '\t')

         #path = "D:/CSV/New"
         #if not os.path.exists(path):
         #    os.makedirs(path)
    snp_list = []
    row_list = []
    trigger = False
    for row in tqdm(reader):
        with open('/home/cappelchi/ReichLab/V42/long/v42.4.1240K.selected_v38.vcf', 'a') as output_file:
            writer = csv.writer(output_file, delimiter = '\t')

            if trigger:
                row_list.append(row)
                snp_list.append(row[2])
                cnt += 1
                if cnt >= chunk:
                    response = Entrez.efetch(db='SNP', id=','.join(snp_list), rettype='flt', retmode='flt').read()
                    dict_response = get_response(response)
                    for rw in row_list:                    
                        rw[1] = dict_response[rw[2]]['CHRPOS'].split(':')[1]
                        writer.writerow(rw)
                    cnt = 0
            else:
                writer.writerow(row)
            if row[0] == '#CHROM':
                trigger = True

         #    writer.writerow(row)
        #cnt += 1
        #if cnt > 35:
            #break


#csvfile.close()

In [65]:
for line in row_list:
#    if line[3]== 'snp_1_1596979':
#        print (line)
    print(line[0], line[1], line[2], line[3])

1 817186 rs3094315 A
1 841166 rs12124819 A
1 897538 rs28765502 T
1 906633 rs7419119 T
1 911484 rs950122 G
1 933923 rs113171913 C
1 955641 rs13302957 A
1 958082 rs59986066 T
1 960891 rs112905931 C
1 968046 rs6696609 C
1 979472 rs13303368 C
1 1014274 rs8997 G
1 1068249 rs4075116 T
1 1070426 rs3934834 C
1 1083324 rs9442372 G
1 1086035 rs3737728 G
1 1086278 rs11260588 G
1 1086315 rs9442398 G
1 1095185 rs6687776 C
1 1096160 rs9651273 G
1 1109951 rs147606383 G
1 1113575 rs4970405 A
1 1114570 rs12726255 A
1 1124855 rs7540009 G
1 1125786 rs11807848 T
1 1127258 rs9442373 A
1 1129599 rs2298217 C
1 1130649 rs12145826 G
1 1141684 rs4970357 A
1 1143818 rs11260603 T
1 1152303 rs9442380 C
1 1155177 rs7553429 A
1 1159358 rs4970362 G
1 1163962 rs9660710 C
1 1170732 rs6670693 A
1 1171093 rs4970420 A
1 1173257 rs4970421 G
1 1184478 rs1320565 C
1 1185051 rs1320571 G
1 1186414 rs11260549 G
1 1193742 rs9659213 G
1 1199862 rs9729550 A
1 1208277 rs3753348 C
1 1215920 rs3813200 C
1 1217251 rs11721 C
1 1220751 

1 3442144 snp_1_3442144 C
1 3445911 rs883427 G
1 3446813 rs4648392 G
1 3448867 rs55762450 C
1 3450637 rs10797400 C
1 3451005 rs12119711 C
1 3454592 rs9970944 G
1 3455757 rs2821066 G
1 3456592 rs4648524 T
1 3456871 rs10737192 G
1 3457295 rs947347 A
1 3464451 rs6685801 T
1 3466713 rs869697 G
1 3472397 rs7516644 A
1 3472928 rs878063 T
1 3473345 rs9628616 C
1 3473428 rs79386324 C
1 3474626 rs2821063 G
1 3480149 rs10909969 C
1 3482334 rs2821059 C
1 3482491 rs79241523 T
1 3482588 rs12749547 C
1 3487126 rs947354 C
1 3491107 rs4431782 T
1 3493819 rs7513251 C
1 3493850 rs2821054 T
1 3495541 rs2821053 C
1 3504073 rs12139206 C
1 3507861 rs4648527 A
1 3510633 rs6697749 C
1 3511295 rs7544357 A
1 3512516 rs2821025 C
1 3512800 rs2821023 C
1 3513966 rs6690231 C
1 3518488 rs60957843 G
1 3520871 rs2821017 G
1 3526417 rs78000706 G
1 3526711 rs59839028 T
1 3531595 rs4648398 T
1 3532643 rs79603637 A
1 3533564 rs55728961 C
1 3537362 rs4648400 G
1 3540374 rs4648537 C
1 3543493 rs4648538 C
1 3544154 rs7404868

In [ ]:
!wget https://reichdata.hms.harvard.edu/pub/datasets/amh_repo/curated_releases/V42/V42.4/SHARE/public.dir/v42.4.1240K_HO.tar

In [ ]:
!mkdir -p /home/cappelchi/ReichLab/V42/long/
!tar -xf v42.4.1240K_HO.tar -C /home/cappelchi/ReichLab/V42/long

In [ ]:
!echo "genotypename:    /home/cappelchi/ReichLab/V42/short/v42.4.1240K_HO.geno" > /home/cappelchi/ReichLab/V42/long/par.EIGENSTRAT.PEDPACKED
!echo "snpname:        /home/cappelchi/ReichLab/V42/short/v42.4.1240K_HO.snp" >> /home/cappelchi/ReichLab/V42/long/par.EIGENSTRAT.PEDPACKED
!echo "indivname:       /home/cappelchi/ReichLab/V42/short/v42.4.1240K_HO.ind" >> /home/cappelchi/ReichLab/V42/long/par.EIGENSTRAT.PEDPACKED
!echo "outputformat:    PACKEDPED" >> /home/cappelchi/ReichLab/V42/long/par.EIGENSTRAT.PEDPACKED
!echo "genotypeoutname: /home/cappelchi/ReichLab/V42/short/v42.4.1240K_HO.bed" >> /home/cappelchi/ReichLab/V42/long/par.EIGENSTRAT.PEDPACKED
!echo "snpoutname:      /home/cappelchi/ReichLab/V42/short/v42.4.1240K_HO.bim" >> /home/cappelchi/ReichLab/V42/long/par.EIGENSTRAT.PEDPACKED
!echo "indivoutname:    /home/cappelchi/ReichLab/V42/short/v42.4.1240K_HO.fam" >> /home/cappelchi/ReichLab/V42/long/par.EIGENSTRAT.PEDPACKED
!echo "familynames:     NO" >> /home/cappelchi/ReichLab/V42/long/par.EIGENSTRAT.PEDPACKED
!cat /home/cappelchi/ReichLab/V42/short/par.EIGENSTRAT.PEDPACKED

In [ ]:
!convertf -p /home/cappelchi/ReichLab/V42/short/par.EIGENSTRAT.PEDPACKED

In [80]:
!head -35 /home/cappelchi/ReichLab/V42/long/v42.4.1240K.selected_v38.vcf

In [19]:
!wget http://hgdownload.cse.ucsc.edu/gbdb/hg19/liftOver/hg19ToHg38.over.chain.gz
!wget http://hgdownload.cse.ucsc.edu/gbdb/hg18/liftOver/hg18ToHg38.over.chain.gz
!wget http://hgdownload.cse.ucsc.edu/gbdb/hg17/liftOver/hg17ToHg19.over.chain.gz

--2020-04-02 08:08:37--  http://hgdownload.cse.ucsc.edu/gbdb/hg18/liftOver/hg18ToHg38.over.chain.gz
Resolving hgdownload.cse.ucsc.edu (hgdownload.cse.ucsc.edu)... 128.114.119.163
Connecting to hgdownload.cse.ucsc.edu (hgdownload.cse.ucsc.edu)|128.114.119.163|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 343575 (336K) [application/x-gzip]
Saving to: ‘hg18ToHg38.over.chain.gz’

hg18ToHg38.over.cha 100%[===================>] 335.52K   114KB/s    in 3.0s    

2020-04-02 08:08:41 (114 KB/s) - ‘hg18ToHg38.over.chain.gz’ saved [343575/343575]

--2020-04-02 08:08:41--  http://hgdownload.cse.ucsc.edu/gbdb/hg17/liftOver/hg17ToHg19.over.chain.gz
Resolving hgdownload.cse.ucsc.edu (hgdownload.cse.ucsc.edu)... 128.114.119.163
Connecting to hgdownload.cse.ucsc.edu (hgdownload.cse.ucsc.edu)|128.114.119.163|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 150239 (147K) [application/x-gzip]
Saving to: ‘hg17ToHg19.over.chain.gz’

hg17ToHg19.over.cha 100%

In [73]:
with open('/home/cappelchi/ReichLab/V42/long/v42.4.1240K.selected.vcf', 'r', encoding='utf-8') as input_file:

    for line in input_file:
        print(line)
        break
with open('/home/cappelchi/ReichLab/V42/long/v42.4.1240K.selected_v38.vcf', 'r', encoding='utf-8') as input_file:
    for line in input_file:
        print(line)
        break

##fileformat=VCFv4.2

##fileformat=VCFv4.2

